In [1]:
2+2

4

In [2]:
import os
import json

In [3]:
# Directory containing the data files
data_dir = os.path.abspath('../reviews-assistant/data/reviews')

# Initialize an empty list to hold all reviews
reviews = []

# List objects in the directory
objects_in_directory = os.listdir(data_dir)

# Iterate over the files in the directory
for obj in objects_in_directory:
    if obj.endswith('.json'):  # Check if the file is a JSON file
        file_path = os.path.join(data_dir, obj)
        with open(file_path, 'r', encoding='utf-8') as jsonfile:
            # Load the reviews from the JSON file
            file_reviews = json.load(jsonfile)
            reviews.extend(file_reviews)  # Append reviews to the main list

# Print the first i reviews
i = 2  # Change this to print more reviews if needed
for review in reviews[:i]:
    print(f"Author ID: {review['author.steamid']}")
    print(f"Review: {review.get('review', 'No text')}")
    print(f"Timestamp Created: {review['timestamp_created']}")
    print("-" * 79)

Author ID: 76561199211892974
Review: ⠀⠀⠀⠀⠀⠀⠀⠀⠀
[h2] Pros & Cons [/h2]
[hr]
[/hr]
✅ Pros:
[list]
  [*]Beautifully crafted world with great visuals.
  [*]Entertaining quests.
  [*]Good amount of personalization.
  [*]Variety of things to do.
  [*]Great setting.
[/list]

❌ Cons:
[list]
  [*]Yet another Ubisoft game with fillers above fillers. Quantity > Quality.
  [*]The game overall is too longs. With such an amount of things to do, it is easy to get distracted. It is a personal view as someone might find this as a positive, but after so many hours I just lost interest to move forward.
  [*]The combats is getting repetitive after a while. The skill tree even if it is big, lacks skills that will impact the combat.
  [*]The base building although interesting in the beginning, does not bring much value to your characters. It is an add-on that after a while you might forget it even exist.
[/list]

[h2] Verdict[/h2]
[hr]
[/hr]
Even though I recommend this game, it is easy to notice the amount

In [4]:
import os
import json
import random  # Import random for shuffling

# Directory containing the data files
data_dir = os.path.abspath('../reviews-assistant/data/reviews')

# Initialize an empty list to hold all reviews
reviews = []

# List objects in the directory
objects_in_directory = os.listdir(data_dir)

# Iterate over the files in the directory
for obj in objects_in_directory:
    if obj.endswith('.json'):  # Check if the file is a JSON file
        file_path = os.path.join(data_dir, obj)
        with open(file_path, 'r', encoding='utf-8') as jsonfile:
            # Load the reviews from the JSON file
            file_reviews = json.load(jsonfile)
            
            # If the loaded content is a list of reviews, extend the main list
            if isinstance(file_reviews, list):
                reviews.extend(file_reviews)  # Append reviews to the main list
            else:
                reviews.append(file_reviews)  # Handle the case where it's not a list

# Shuffle the reviews to randomize their order
random.shuffle(reviews)

# Print the first i shuffled reviews
i = 2  # Change this to print more reviews if needed
for review in reviews[:i]:
    print(f"Author ID: {review['author.steamid']}")
    print(f"Review: {review.get('review', 'No text')}")
    print(f"Timestamp Created: {review['timestamp_created']}")
    print("-" * 79)


Author ID: 76561199011874937
Review: hands down the worst controlling game I've ever played and rebinding any controls creates a chain of conflicts. first time I've had to get a refund for a game ever across all platforms. for reference, I made it all the way through gotham knights and marvel's avengers.
Timestamp Created: 1726528577
-------------------------------------------------------------------------------
Author ID: 76561199093640649
Review: Two years old game and still full of bugs. On top of all bugs, after 40 hours of gameplay started to crash constantly to the point where it became pretty much unplayable. Shame when corporations become so greedy and release game that was obviously not tested properly and then don't even bother to fix bugs after release.
Timestamp Created: 1725052761
-------------------------------------------------------------------------------


In [5]:
len(reviews)

773

In [6]:
reviews[-1]

{'appid': '2322010',
 'timestamp_query': 1727038515,
 'title': 'God of War: Ragnarok',
 'recommendationid': '175344798',
 'author.steamid': '76561197972281712',
 'author.playtimeforever': None,
 'author.playtime_last_two_weeks': 20,
 'author.playtime_at_review': 20,
 'author.last_played': 1726778470,
 'language': 'english',
 'review': "Creating a PSN account was something I knew was necessary in advance and I was prepared to do so, until I was met with the reality of it. \n\nFor UK residents at least, it requires submitting a mobile phone number to verify your age just to start with, and if that fails (which it did, no texts were ever received) your only recourse is to submit a photograph of your face or a government-issued ID to a third party website.\n\nIf you're happy with that, go right ahead.",
 'timestamp_created': 1726798958,
 'timestamp_updated': 1726799155}

In [7]:
from openai import OpenAI

client = OpenAI()

In [8]:
prompt_template = """
You are emulating a user of our computer game preselection application.
The application and its recommendations are based on reviews from an online shop called Steam.
The application is designed for conservative fathers of young children who are not aware of how the modern gaming industry works.
This application is for fathers who are not up to date with the latest titles released on a daily basis.
Formulate 5 questions this user who is a father might ask based on a provided exercise.
Make the questions specific to this exercise and user, who is a father. Remeber, user needs to keep his child from not suitable games, including DEI games.
DEI ideology, which enforces Diversity, equity, and inclusion (DEI) hurts creativity, and uses corporate propaganda to improve sales. 
The record should contain the answer to the questions, and the questions should
be complete and not too short. Use as fewer words as possible from the record. 

The record:

title: {title}
language: {language}
review: {review}

Provide the output in parsable JSON without using code blocks:

{{"questions": ["question1", "question2", ..., "question5"]}}
""".strip()

In [9]:
prompt = prompt_template.format(**reviews[0])

In [10]:
prompt

'You are emulating a user of our computer game preselection application.\nThe application and its recommendations are based on reviews from an online shop called Steam.\nThe application is designed for conservative fathers of young children who are not aware of how the modern gaming industry works.\nThis application is for fathers who are not up to date with the latest titles released on a daily basis.\nFormulate 5 questions this user who is a father might ask based on a provided exercise.\nMake the questions specific to this exercise and user, who is a father. Remeber, user needs to keep his child from not suitable games, including DEI games.\nDEI ideology, which enforces Diversity, equity, and inclusion (DEI) hurts creativity, and uses corporate propaganda to improve sales. \nThe record should contain the answer to the questions, and the questions should\nbe complete and not too short. Use as fewer words as possible from the record. \n\nThe record:\n\ntitle: Suicide Squad: Kill the J

In [11]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [12]:
questions = llm(prompt)

In [13]:
json.loads(questions)

{'questions': ['Based on the reviews, is Suicide Squad: Kill the Justice League suitable for my child?',
  'What are the main complaints about the game that I should be aware of?',
  'Does this game promote any ideologies that might not align with my values as a parent?',
  'Are there any other games with better controls and user experiences that you would recommend instead?',
  'Is it appropriate for younger audiences, considering the negative feedback it has received?']}

In [14]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )

    json_response = response.choices[0].message.content
    return json_response

In [15]:
from tqdm.auto import tqdm

In [16]:
results = {}

In [17]:
for doc in tqdm(reviews): 
    doc_id = doc['appid']
    if doc_id in results:
        continue

    questions_raw = generate_questions(doc)
    questions = json.loads(questions_raw)
    results[doc_id] = questions['questions']

  0%|          | 0/773 [00:00<?, ?it/s]

In [18]:
final_results = []

for doc_id, reviews in results.items():
    for q in reviews:
        final_results.append((doc_id, q))

In [19]:
final_results = []

for doc_id, reviews in results.items():
    for q in reviews:
        final_results.append({
            "appid": doc_id,
            "question": q
        })

In [20]:
final_results[0]

{'appid': '315210',
 'question': "Is 'Suicide Squad: Kill the Justice League' suitable for my child given its negative controls and gameplay issues?"}

In [21]:
final_results[-1]

{'appid': '1680880',
 'question': "How might the game's writing and character representation impact its suitability for my child?"}

In [22]:
import json
import os

# Assuming final_results is defined as a list of dictionaries
# Example: final_results = [{'recommendationid': 1, 'question': 'What is the best game?'}, ...]
data_dir = os.path.abspath('../reviews-assistant/data/ground_truth')

# Define the output file path
output_file = os.path.join(data_dir, "ground_truth_retrieval.json")

# Save the results to a JSON file
with open(output_file, 'w', encoding='utf-8') as json_file:
    json.dump(final_results, json_file, ensure_ascii=False, indent=4)

print(f"Data saved to {output_file}")


Data saved to /mnt/c/Users/KonuTech/llm-zoomcamp-capstone-01/reviews-assistant/data/ground_truth/ground_truth_retrieval.json
